## Playground for utilities to load, parse, and transform bank files


In [70]:
import pandas as pd
# import numpy as np
import json

from decimal import Decimal # To represent monetary values  https://docs.python.org/3/library/decimal.html  

In [71]:
# Data types for GL data
type TransactionID = str   # Unique identifier for each transaction - TBD / maybe a hash of the transaction details?
type TransactionItemID = int # Item number within transaction document
type TransactionDate = str # ... to be determined how to handle this
type TransactionAmount = Decimal # 
type DebitCreditIndicator = str # 'D' or 'C'
type TransactionDescription = str
type AccountID = str
type CurrencyUnit = str
type BusinessPartner = str
type Origin = str  # The original bank or credit card account that the transaction came from

# Can possibly be simplified by using @dataclass decorator
# Data type of GL item
class GLItem:
    def __init__(self, transactionID: TransactionID, transactionItemID: TransactionItemID, 
                 transactionDate: TransactionDate, transactionAmount: TransactionAmount, 
                 debitCreditIndicator: DebitCreditIndicator, transactionDescription: TransactionDescription, accountID: AccountID, 
                 currencyUnit: CurrencyUnit, businessPartner: BusinessPartner, origin: Origin):
        self.transactionID = transactionID
        self.transactionItemID = transactionItemID
        self.transactionDate = transactionDate
        self.transactionAmount = transactionAmount
        self.debitCreditIndicator = debitCreditIndicator
        self.transactionDescription = transactionDescription
        self.accountID = accountID
        self.currencyUnit = currencyUnit
        self.businessPartner = businessPartner
        self.Origin = origin

    def __str__(self):
        return f'{self.transactionID}, {self.transactionDate}, {self.transactionAmount}, {self.debitCreditIndicator}, {self.transactionDescription}, {self.accountID}, {self.currencyUnit}, {self.businessPartner}, {self.Origin}'

    def __repr__(self):
        return f'{self.transactionID}, {self.transactionDate}, {self.transactionAmount}, {self.debitCreditIndicator}, {self.transactionDescription}, {self.accountID}, {self.currencyUnit}, {self.businessPartner}, {self.Origin}'

In [ ]:
# Read Wells Fargo test file into data frame
wfc_ColumnNames = ['Date', 'Amount', 'Unknown', 'CheckNo', 'Description']
# wfc_ColumnDataTypes = {'Amount': np.float16, 'Unknown': str, 'CheckNo': np.int16, 'Description': str}
wfc_RawData = pd.read_csv('Bank-Files/WF-test.csv', index_col=None, parse_dates=True, 
                         names=wfc_ColumnNames)
wfc_RawData.head(5)

In [ ]:
# Read bank descriptor
with open('BankDescriptor-WFC.json', 'r') as file:
    wfc_bankDescriptor = json.load(file)

print(wfc_bankDescriptor)

In [ ]:
# Read account determination (JSON file) into dictionary

with open(wfc_bankDescriptor['AccountDeterminationFile'], 'r') as file:
    wfc_accountDetermination = json.load(file)

print(wfc_accountDetermination)

In [75]:
# print(wfc_accountDetermination['PITCAIRN HOMEOWN Payment']) #to be updated to match new JSON structure

In [76]:

# for key, value in wfc_accountDetermination.items():
#    print(f"String: {key}, Account: {value["Account"]}")

In [ ]:

for index, row in wfc_RawData.head(5).iterrows():
    print(f"Index: {index}")
    print(row.Date, row.Amount, row.Description)

In [78]:
# Another way to loop the data frame
# Assuming df is your DataFrame
#def process_row(row):
#    print(row)

#wfc_RawData.apply(process_row, axis=1)

In [ ]:
# Loop through the data frame and determine the account
for index, row in wfc_RawData.head(5).iterrows():
    print(f"Index: {index}")
    print(row.Date, row.Amount, row.Description)
    for key, value in wfc_accountDetermination.items():
        #print(f"String: {key}, Account: {value}")
        if key in row.Description:
            print(f"Account determination: String: {key}, Account: {value["Account"]}, Business Partner: {value["BP"]}")
            break
    # handle case when no account is found: ...
    

In [80]:
class AccountDetermination:
    def __init__(self, json_file):
        with open(json_file, 'r') as file:
            self.accountDetermination = json.load(file)

    def determine_account(self, description: str):
        for item in self.accountDetermination['accountDetermination']:
            if item['searchString'] in description:
                return item
        return None

# Example usage:
# account_det = AccountDetermination('AccountDetermination-WFC.json')
# account = account_det.determine_account('INTEREST PAYMENT')
# print(account)

In [ ]:
import hashlib
import sqlite3

#
accountDetermination_wfc = AccountDetermination('AccountDetermination-WFC.json')

transactionItemID: TransactionItemID

for index, row in wfc_RawData.head(15).iterrows():
    print(f"Index: {index}", row.Date, row.Amount, row.Description, 
          f"Account: {accountDetermination_wfc.determine_account(row.Description)}")
    accounts = accountDetermination_wfc.determine_account(row.Description)
    if accounts is None:
        print(f"Account not found for description: {row.Description}")
        continue
    accountID = accounts['Account']
    businessPartner = accounts['BP']
    print(f"Account: {accountID}, Business Partner: {businessPartner}")
    transactionDate = row.Date
    transactionAmount: TransactionAmount = row.Amount
    print(f"Transaction Date: {transactionDate}, Transaction Amount: {transactionAmount}")
    transactionDescription = row.Description
    print(f"Transaction Description: {transactionDescription}")
    # set the currency unit
    currencyUnit = 'USD'
    # Set the debit or credit indicator (this is based on GL, no how banks handle it - just the opposite)
    # For a regular bank account, a deposit is a debit and a withdrawal is a credit to the bank balance sheet account
    if transactionAmount >= 0:
        debitCreditIndicator = 'D' # Debit to bank balance sheet account
    else:
        debitCreditIndicator = 'C' # Credit to bank balance sheet account
    # Set the origin
    Origin = wfc_bankDescriptor['BankAccountDescription']

    # set the transactionID
    # Create a hash of the complete row to use as transactionID
    row_data = f"{row.Date}{row.Amount}{row.Description}{Origin}"
    transactionID = hashlib.sha256(row_data.encode()).hexdigest()
    transactionItemID = 1
    print(f"Transaction ID: {transactionID}")

    # Create a GL item
    gl_item = GLItem(transactionID, transactionItemID, transactionDate, transactionAmount, debitCreditIndicator, transactionDescription, accountID, currencyUnit, businessPartner, Origin)
    print("--------------------")
    print(gl_item)
    print("--------------------")
    # write the GL item to a file
    #with open('GL-Files/gl_items.csv', 'a') as f:
    #    f.write(f'{gl_item.transactionID},{gl_item.transactionDate},{gl_item.transactionAmount},{gl_item.debitCreditIndicator},{gl_item.transactionDescription},{gl_item.accountID},{gl_item.currencyUnit},{gl_item.businessPartner},{gl_item.Origin}\n')

    # Connect to SQLite database (or create it if it doesn't exist)
    conn = sqlite3.connect('GL-Files/gl_items.db')
    cursor = conn.cursor()

    # Create table if it doesn't exist
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS gl_items2 (
        transactionID TEXT,
        transactionItemID INTEGER,
        transactionDate TEXT,
        transactionAmount REAL,
        debitCreditIndicator TEXT,
        transactionDescription TEXT,
        accountID TEXT,
        currencyUnit TEXT,
        businessPartner TEXT,
        origin TEXT,
        PRIMARY KEY (transactionID, transactionItemID)
    )
    ''')

    # Check if the transactionID already exists in the database
    cursor.execute('SELECT transactionID FROM gl_items2 WHERE transactionID = ?', (transactionID,))
    if cursor.fetchone() is None:
        # Insert the GL item into the database
        cursor.execute('''
        INSERT INTO gl_items2 (transactionID, transactionItemID, transactionDate, transactionAmount, debitCreditIndicator, transactionDescription, accountID, currencyUnit, businessPartner, origin)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (gl_item.transactionID, gl_item.transactionItemID, gl_item.transactionDate, gl_item.transactionAmount, gl_item.debitCreditIndicator, gl_item.transactionDescription, gl_item.accountID, gl_item.currencyUnit, gl_item.businessPartner, gl_item.Origin))
        conn.commit()
        print(f"Inserted GL item with transactionID: {transactionID}")
    else:
        print(f"GL item with transactionID: {transactionID} already exists in the database")

    # Close the database connection
    conn.close()